In [1]:
import geopandas as gpd
import folium
from folium.features import GeoJsonPopup
from ipywidgets import widgets, interact
from IPython.display import display
from branca.element import Template, MacroElement
from branca.colormap import linear
from ipywidgets import interact, Dropdown, HBox
import branca
from folium import IFrame
from branca.element import Figure
# Charger le fichier shapefile
data = gpd.read_file("C:/Users/CE PC/Desktop/Elections_2024/departement/departement.shp")
# Fonction pour mettre à jour la carte en fonction de la sélection
def update_map(Information, Departement=''):
    # Reconstruire la carte
    carte = folium.Map(location=[14.4, -14.7], zoom_start=7)
    folium.TileLayer('cartodbpositron').add_to(carte)
    
    # Définition des couleurs pour chaque couche
    colors = {
        'electeurs': 'YlGn',
        'bureaux_de': 'YlOrRd',
        'lieux_de_v': 'PuBu'
    }

    # Définition des textes de légende pour chaque couche
    legend_texts = {
        'electeurs': 'Nombre d\'électeurs par département',
        'bureaux_de': 'Nombre de bureaux de vote par département',
        'lieux_de_v': 'Nombre de centres de vote par département'
    }

    # Ajouter la nouvelle couche choropleth avec la couleur et le texte de légende correspondants
    choropleth = folium.Choropleth(
        geo_data=data,
        name=Information,
        data=data,
        columns=['DEPT', Information],
        key_on='feature.properties.DEPT',
        fill_color=colors[Information],  # Utiliser la couleur définie pour la couche
        fill_opacity=1,
        line_opacity=0,
        legend_name=legend_texts[Information],  # Utiliser le texte de légende défini pour la couche
    ).add_to(carte)
    # Définir un dictionnaire de libellés personnalisés
    labels_map = {
        'electeurs': 'Nombre d\'électeurs',
        'bureaux_de': 'Bureaux de vote',
        'lieux_de_v': 'Centres de vote'
    }
    # Ajouter des popups pour chaque département
    folium.GeoJson(
        data,
        name='Popup Info',
        style_function=lambda x: {'color': 'black', 'weight': 0.5},
        tooltip=folium.GeoJsonTooltip(
            fields=['DEPT', Information],
            aliases=['Département', labels_map[Information]], 
            localize=True,
            sticky=False,
            labels=True,
            style="""
                background-color: #F0EFEF;
                border: 2px solid black;
                border-radius: 3px;
                box-shadow: 3px;
            """,
            max_width=800,
        ),
        popup=folium.GeoJsonPopup(
            fields=['DEPT', Information],
            aliases=['', ''],
            labels=False,
            style='font-size: 15px; font-weight: bold;'
        )
    ).add_to(choropleth.geojson)
    
    # Si un département est sélectionné, extraire les coordonnées du centroïde et zoomer sur ce département
    # Si un département est sélectionné, extraire les coordonnées du centroïde et zoomer sur ce département
    if Departement and Departement in data['DEPT'].unique():
        departement_data = data[data['DEPT'] == Departement].iloc[0]
        centroid = departement_data.geometry.centroid
        carte.location = [centroid.y, centroid.x]
        carte.zoom_start = 12
        # Créer une fenêtre popup avec les informations du département
        html = f"""
        <h4>{Departement}</h4>
        <ul>
        <li>Nombre d'électeurs: {departement_data['electeurs']}</li>
        <li>Bureaux de vote: {departement_data['bureaux_de']}</li>
        <li>Centres de vote: {departement_data['lieux_de_v']}</li>
        </ul>
        <script>
            document.addEventListener('DOMContentLoaded', function() {{
                var popup = document.querySelector('.popup-content');
                popup.style.display = 'block';
            }});
        </script>
        """
        iframe = IFrame(html=html, width=220, height=150)
        popup = folium.Popup(iframe, max_width=250, show=True)  # Utiliser show=True pour montrer la popup par défaut
        folium.Marker([centroid.y, centroid.x], popup=popup).add_to(carte)

        # Ajouter l'effet clignotant sur le département sélectionné
        def style_function(feature):
            return {
                'fillColor': '#FF0000',
                'fillOpacity': 0.5,
                'className': 'blinking'
            }
        folium.GeoJson(
            departement_data.geometry,
            style_function=style_function
        ).add_to(carte)


    
    # Mettre à jour la légende
    carte.add_child(folium.LayerControl())
    
    # Retourner l'objet carte à la fin de la fonction
    return carte


# Créer le menu déroulant pour les informations
dropdown_information = Dropdown(
    options={
        'Nombre d\'électeurs': 'electeurs', 
        'Nombre de bureaux de vote': 'bureaux_de', 
        'Nombre de centres de vote': 'lieux_de_v'
    },
    value='electeurs',
    description='Information:',
)

# Créer le menu déroulant pour la sélection des départements
dropdown_departement = Dropdown(
    options=['Sélectionner un département'] + list(data['DEPT'].unique()),
    value='Sélectionner un département',  # Définir la valeur par défaut
    description='Département:',
)


# Utiliser HBox pour placer les menus déroulants sur la même ligne
ui = HBox([dropdown_information, dropdown_departement])

# Afficher les menus déroulants et la carte mise à jour en fonction de la sélection
interact(update_map, Information=dropdown_information, Departement=dropdown_departement)


interactive(children=(Dropdown(description='Information:', options={"Nombre d'électeurs": 'electeurs', 'Nombre…

<function __main__.update_map(Information, Departement='')>